# Aim

1) Cluster article titles in the "Towards Data Science" publication, to find clusters that make the most sense

2) Find which clusters (word themes) lead to the largest numbers of claps. Possibly use (claps / number of followers) in order to normalize for the number of followers that one already has. Though, followers can be a lot of dead weight that then penalize posters that are too popular a priori (think Youtube subscribers - not all active)

3) **Make a "Data Science article title generator" (speak to Aaron)**

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

from pandas_profiling import ProfileReport

In [3]:
df = pd.read_csv('Medium_AggregatedData.csv')

In [4]:
df.shape

(279577, 50)

In [5]:
pd.options.display.max_columns = 50
df.head()

,audioVersionDurationSec,codeBlock,codeBlockCount,collectionId,createdDate,createdDatetime,firstPublishedDate,firstPublishedDatetime,imageCount,isSubscriptionLocked,language,latestPublishedDate,latestPublishedDatetime,linksCount,postId,readingTime,recommends,responsesCreatedCount,socialRecommendsCount,subTitle,tagsCount,text,title,totalClapCount,uniqueSlug,updatedDate,updatedDatetime,url,vote,wordCount,publicationdescription,publicationdomain,publicationfacebookPageName,publicationfollowerCount,publicationname,publicationpublicEmail,publicationslug,publicationtags,publicationtwitterUsername,tag_name,slug,name,postCount,author,bio,userId,userName,usersFollowedByCount,usersFollowedCount,scrappedDate
0,0,NaN,0.0,638f418c8464,2018-09-18,2018-09-18 20:55:34,2018-09-18,2018-09-18 20:57:03,1,False,en,2018-09-18,2018-09-18 20:58:20,1,10007d3018fe,0.958491,2,0,0,A major private IT company implements blockcha...,5,"Private Business, Government and Blockchain\n\...","Private Business, Government and Blockchain",100,private-business-government-and-blockchain-100...,2018-09-18,2018-09-18 20:58:20,https://medium.com/s/story/private-business-go...,False,201,ICOBox is the first and the biggest new genera...,NaN,icobox.io,NaN,ICOBox,support@icobox.io,icobox-io,"BLOCKCHAIN,ICO,ETHEREUM,ETHEREUM BLOCKCHAIN,TO...",icobox_io,Blockchain,blockchain,Blockchain,265164.0,Anar Babaev,NaN,f1ad85af0169,babaevanar,450.0,404.0,20181104
1,0,NaN,0.0,638f418c8464,2018-09-18,2018-09-18 20:55:34,2018-09-18,2018-09-18 20:57:03,1,False,en,2018-09-18,2018-09-18 20:58:20,1,10007d3018fe,0.958491,2,0,0,A major private IT company implements blockcha...,5,"Private Business, Government and Blockchain\n\...","Private Business, Government and Blockchain",100,private-business-government-and-blockchain-100...,2018-09-18,2018-09-18 20:58:20,https://medium.com/s/story/private-business-go...,False,201,ICOBox is the first and the biggest new genera...,NaN,icobox.io,NaN,ICOBox,support@icobox.io,icobox-io,"BLOCKCHAIN,ICO,ETHEREUM,ETHEREUM BLOCKCHAIN,TO...",icobox_io,Samsung,samsung,Samsung,5708.0,Anar Babaev,NaN,f1ad85af0169,babaevanar,450.0,404.0,20181104
2,0,NaN,0.0,638f418c8464,2018-09-18,2018-09-18 20:55:34,2018-09-18,2018-09-18 20:57:03,1,False,en,2018-09-18,2018-09-18 20:58:20,1,10007d3018fe,0.958491,2,0,0,A major private IT company implements blockcha...,5,"Private Business, Government and Blockchain\n\...","Private Business, Government and Blockchain",100,private-business-government-and-blockchain-100...,2018-09-18,2018-09-18 20:58:20,https://medium.com/s/story/private-business-go...,False,201,ICOBox is the first and the biggest new genera...,NaN,icobox.io,NaN,ICOBox,support@icobox.io,icobox-io,"BLOCKCHAIN,ICO,ETHEREUM,ETHEREUM BLOCKCHAIN,TO...",icobox_io,It,it,It,3720.0,Anar Babaev,NaN,f1ad85af0169,babaevanar,450.0,404.0,20181104
3,0,NaN,0.0,NaN,2018-01-07,2018-01-07 17:04:37,2018-01-07,2018-01-07 17:06:29,13,False,en,2018-01-07,2018-01-07 17:18:38,24,1000c43bcb97,19.716981,0,0,0,Introduction,5,EPQ draft 1 (4844 words)\nhttps://upload.wikim...,EPQ draft 1 (4844 words),0,introduction-3-1000c43bcb97,2018-01-07,2018-01-07 17:18:39,https://medium.com/s/story/introduction-3-1000...,False,4854,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Technology,technology,Technology,166125.0,George Sykes,NaN,93b9e94f08ca,tasty231,6.0,22.0,20181104
4,0,NaN,0.0,NaN,2018-01-07,2018-01-07 17:04:37,2018-01-07,2018-01-07 17:06:29,13,False,en,2018-01-07,2018-01-07 17:18:38,24,1000c43bcb97,19.716981,0,0,0,Introduction,5,EPQ draft 1 (4844 words)\nhttps://upload.wikim...,EPQ draft 1 (4844 words),0,introduction-3-1000c43bcb97,2018-01-07,2018-01-07 17:18:39,https://medium.com/s/story/introduction-3-1000...,False,4854,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Robotics,robotics,Robotics,9103.0,George Sykes,NaN,93b9e94f08ca,tasty231,6.0,22.0,20181104


In [6]:
pd.options.display.max_columns = 20
pd.options.display.max_rows = 20

In [7]:
df.columns

Index(['audioVersionDurationSec', 'codeBlock', 'codeBlockCount',
       'collectionId', 'createdDate', 'createdDatetime', 'firstPublishedDate',
       'firstPublishedDatetime', 'imageCount', 'isSubscriptionLocked',
       'language', 'latestPublishedDate', 'latestPublishedDatetime',
       'linksCount', 'postId', 'readingTime', 'recommends',
       'responsesCreatedCount', 'socialRecommendsCount', 'subTitle',
       'tagsCount', 'text', 'title', 'totalClapCount', 'uniqueSlug',
       'updatedDate', 'updatedDatetime', 'url', 'vote', 'wordCount',
       'publicationdescription', 'publicationdomain',
       'publicationfacebookPageName', 'publicationfollowerCount',
       'publicationname', 'publicationpublicEmail', 'publicationslug',
       'publicationtags', 'publicationtwitterUsername', 'tag_name', 'slug',
       'name', 'postCount', 'author', 'bio', 'userId', 'userName',
       'usersFollowedByCount', 'usersFollowedCount', 'scrappedDate'],
      dtype='object')

In [8]:
df.loc[df['language'] == 'en', 'publicationname'].value_counts()

Towards Data Science                                16584
Hacker Noon                                          5540
Becoming Human: Artificial Intelligence Magazine     3082
Data Driven Investor                                 2823
Chatbots Life                                        2041
Chatbots Magazine                                    1439
SyncedReview                                         1430
The Startup                                          1383
Good Audience                                        1058
freeCodeCamp.org                                     1005
                                                    ...  
Loon Blog                                               1
mltutorials.tensorflow                                  1
Aardvark Spaghetti                                      1
Virtual Revolution                                      1
applab                                                  1
Radius-Engineering                                      1
Autonomous Des

In [9]:
# The final columns that I will want to filter down to, once I have trained my unsupervised learning algorithm

# COLUMN OF NOTE: 'recommends' is the number of unique users who clapped

#df = df[['postId','createdDate','language','title','totalClapCount','recommends','publicationname',
#         'usersFollowedCount']]


I'm now looking for a way to see what columns are causing the duplicate rows, and if I should incorporate them into my final df, or if I should eliminate the duplicates and just take the first option. I think eliminating duplicate post_ID's right now makes the most sense, and I can think about what kind of extra columns I want later. Maybe each unique post_ID is all that matters? Eliminate all duplicates of post_ID

In [10]:
#nlp_df = df.loc[row_logic, ['postId', 'title','createdDate']]

In [11]:
df[df['postId'] == 'c3d8b6551bbe']

,audioVersionDurationSec,codeBlock,codeBlockCount,collectionId,createdDate,createdDatetime,firstPublishedDate,firstPublishedDatetime,imageCount,isSubscriptionLocked,...,slug,name,postCount,author,bio,userId,userName,usersFollowedByCount,usersFollowedCount,scrappedDate
170290,0,NaN,0.0,NaN,2017-12-08,2017-12-08 12:54:22,2017-12-12,2017-12-12 22:46:30,12,False,...,data-science,Data Science,33617.0,Rafael Coelho,Ciências Sociais (USP/FFLCH),be9b2bc5d3af,rafacastrocoelhosilva,5.0,3.0,20181104
170291,0,NaN,0.0,NaN,2017-12-08,2017-12-08 12:54:22,2017-12-12,2017-12-12 22:46:30,12,False,...,r,R,1558.0,Rafael Coelho,Ciências Sociais (USP/FFLCH),be9b2bc5d3af,rafacastrocoelhosilva,5.0,3.0,20181104
170292,0,NaN,0.0,NaN,2017-12-08,2017-12-08 12:54:22,2017-12-12,2017-12-12 22:46:30,12,False,...,github,Github,7846.0,Rafael Coelho,Ciências Sociais (USP/FFLCH),be9b2bc5d3af,rafacastrocoelhosilva,5.0,3.0,20181104
170293,0,NaN,0.0,NaN,2017-12-08,2017-12-08 12:54:22,2017-12-12,2017-12-12 22:46:30,12,False,...,universidade,Universidade,948.0,Rafael Coelho,Ciências Sociais (USP/FFLCH),be9b2bc5d3af,rafacastrocoelhosilva,5.0,3.0,20181104
170294,0,NaN,0.0,NaN,2017-12-08,2017-12-08 12:54:22,2017-12-12,2017-12-12 22:46:30,12,False,...,docentes,Docentes,150.0,Rafael Coelho,Ciências Sociais (USP/FFLCH),be9b2bc5d3af,rafacastrocoelhosilva,5.0,3.0,20181104
170295,0,NaN,0.0,NaN,2017-12-08,2017-12-08 12:54:22,2017-12-12,2017-12-12 22:46:30,12,False,...,usp,Usp,240.0,Rafael Coelho,Ciências Sociais (USP/FFLCH),be9b2bc5d3af,rafacastrocoelhosilva,5.0,3.0,20181104


In [12]:
row_logic = ((df['language'] == 'en') & (df['publicationname'] == 'Towards Data Science'))

row_logic

0         False
1         False
2         False
3         False
4         False
5         False
6         False
7         False
8         False
9         False
          ...  
279567    False
279568    False
279569    False
279570    False
279571    False
279572    False
279573    False
279574    False
279575    False
279576    False
Length: 279577, dtype: bool

In [13]:
df_filt = df[row_logic].drop_duplicates(subset = 'postId')['title'] # Filtering for just the unique post_ids
df_filt.head()

1485    Published previously for Venturebeat and the Z...
2829    Inferring the effect of Brand Search using Cau...
3134                    4 WAYS TO SUM DATA BY WEEK NUMBER
5268    AR, AI, Ay Yi Yi: 2 Takeaways from NYC Media L...
5448                Engineer’s Guide to Julia Programming
Name: title, dtype: object

**Note: I need to keep the index the same so that I can join on the index later on when I am done clustering my data**

# Pickle Series of Titles

In [14]:
df_filt.to_pickle('titles.pkl')

In [15]:
df.to_pickle('df.pkl')

In [16]:
#titles = pd.read_pickle('titles.pkl')